In [2]:
!python --version

Python 3.7.11


In [ ]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
??verify_images

# From Model to Production

## The Practice of Deep Learning

### Starting Your Project

### The State of Deep Learning

#### Computer vision

#### Text (natural language processing)

#### Combining text and images

#### Tabular data

#### Recommendation systems

#### Other data types

### The Drivetrain Approach

## Gathering Data

# clean
To download images with Bing Image Search, sign up at [Microsoft Azure](https://azure.microsoft.com/en-us/services/cognitive-services/bing-web-search-api/) for a free account. You will be given a key, which you can copy and enter in a cell as follows (replacing 'XXX' with your key and executing it):

In [ ]:
import ddg_search

image_urls = ddg_search.search_images_ddg("dachshund")
image_urls = L(image_urls)
image_urls[0]

In [ ]:
dest = Path("dachshund")
if not dest.exists():
    dest.mkdir()

In [ ]:
download_url(image_urls[0], "dachshund/test_dachshund.jpg")

In [ ]:
im = Image.open("dachshund/test_dachshund.jpg")
im.to_thumb(128,128)

In [ ]:
download_images(dest, urls=image_urls)

In [ ]:
fns = get_image_files(dest)
type(fns)

In [ ]:
fns

In [ ]:
bear_fns = get_image_files("../bears")
fns.extend(bear_fns)

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

### Sidebar: Getting Help in Jupyter Notebooks

### End sidebar

## From Data to DataLoaders

In [ ]:
doc(TensorImage)

In [ ]:
# Testing transformations

img = Image.open(fns[0])
# im_t = cast(array(img), TensorImage)

timg = TensorImage(array(img)).permute(2,0,1).float()/255.
def _batch_ex(bs): return TensorImage(timg[None].expand(bs, *timg.shape).clone())

imgs = _batch_ex(5)
# deflt = Flip()
# show_images( deflt(imgs) ,suptitle='Default Flip')

# tfm()

transformed = []

for tfm in tfms:
#     print(f'item = {item}. type = {type(item)}\n')
    transformed_images = tfm(imgs)
    for image in transformed_images:
        transformed.append(image)

show_images(transformed)

In [ ]:
# The ImageDataLoaders allow one to load data and transform it.
# item_tfms : All images need to be same size. Randomly snapshot different sections of the same image instead of other
#             methods which will cause loss of information
# seed : Affects train/validation split.
# batch_tfms
block = DataBlock(blocks = (ImageBlock,CategoryBlock),
                  get_items=get_image_files,
                  splitter = RandomSplitter(seed=42),
                  get_y=parent_label,
                  item_tfms=RandomResizedCrop(224))

block = block.new(item_tfms=RandomResizedCrop(224, min_scale=0.5), batch_tfms=aug_transforms(mult=2))

In [ ]:
dls = block.dataloaders("images")

In [ ]:
doc(DataBlock)

In [ ]:
doc(ImageDataLoaders.from_folder)

In [ ]:
dls.valid.show_batch(max_n=8, nrows=2)

### Data Augmentation

## Training Your Model, and Using It to Clean Your Data

In [ ]:
learner = cnn_learner(dls, resnet34, metrics=error_rate)
learner.fine_tune(4)

In [ ]:
fns[0].parent.name

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=5)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
#hide
# for idx in cleaner.delete(): cleaner.fns[idx].unlink()
# for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

## Turning Your Model into an Online Application

### Using the Model for Inference

In [ ]:
learner.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
learn_inf.predict('../images/grizzly.jpg')

In [ ]:
learn_inf.dls.vocab

### Creating a Notebook App from the Model

In [ ]:
btn_upload = FileUpload()
btn_upload

In [ ]:
#hide
# For the book, we can't actually click an upload button, so we fake it
btn_upload = SimpleNamespace(data = ['images/grizzly.jpg'])

In [ ]:
img = PILImage.create(btn_upload.data[-1])

In [ ]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl:
    display(img.resize((224,224)))
out_pl

In [ ]:
pred,pred_idx,probs = learn_inf.predict(img)
print(f'Prediction: {pred}; pred_idx = {pred_idx}; probs = {probs}')
f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [ ]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediection: {pred}; Probability: {probs[pred_idx]:0.04f}'
lbl_pred

In [ ]:
btn_run = widgets.Button(description="Classify")
btn_run

In [ ]:
def on_click_classify():
    img = btn_upload.data[-1]
    out_pl.clear_output()
    with out_pl:
        display(img.resize(224,224))
    prediction, pred_idx, probs = learn_inf.predict(img)
    lbl_pred.value = f'prediction = {prediction}; Probability = {pros[pred_idx]:0.04f}'
    
btn_run.on_click(on_click_classify)

In [ ]:
#hide
#Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()

In [ ]:
VBox([widgets.Label("Select your dashchund"), btn_upload,  btn_run, out_pl, lbl_pred])

### Turning Your Notebook into a Real App

In [ ]:
#hide
# !pip install voila
# !jupyter serverextension enable --sys-prefix voila 

### Deploying your app

## How to Avoid Disaster

### Unforeseen Consequences and Feedback Loops

## Get Writing!

## Questionnaire

1. Provide an example of where the bear classification model might work poorly in production, due to structural or style differences in the training data.
Printed image for example

1. Where do text models currently have a major deficiency?
while classification is very good with text models, summarizing, translation or generation of correct text is not very good. For example, right now there's no good way to generate accurate medical respones/information.

1. What are possible negative societal implications of text generation models?
It's very easy to generate contextually appropriate text, but hard to recognize when text has been generated by a computer or whether text is correct. Therefore it can be used to spread disinformation

1. In situations where a model might make mistakes, and those mistakes could be harmful, what is a good alternative to automating a process?
Pairing model with human to make human more efficient.

1. What kind of tabular data is deep learning particularly good at?
natural language in columns or high cardinality categorical variables such as zip code

1. What's a key downside of directly using a deep learning model for recommendation systems?
take longer to train

1. What are the steps of the Drivetrain Approach?
Start with _clear_ objective, then look at _levers_ you can pull to better achieve that objective, then consider the _data_ needed for achieving objective.

1. How do the steps of the Drivetrain Approach map to a recommendation system?

1. Create an image recognition model using data you curate, and deploy it on the web.

1. What is `DataLoaders`?
`DataLoaders` is a light-weight class that stores all the `DataLoader` objects making them available as `train` and `valid`

1. What four things do we need to tell fastai to create `DataLoaders`?
- Where to load data from
- how to label data
- train/validation split
- what type of data it is


1. What does the `splitter` parameter to `DataBlock` do?
split to train/validation

1. How do we ensure a random split always gives the same validation set?
`seed`

1. What letters are often used to signify the independent and dependent variables?
`y` and `x` 

1. What's the difference between the crop, pad, and squish resize approaches? When might you choose one over the others?
`crop` randomly crops a section of the image
`pad` will pad with black/white space
`squish` will squish the image to make it fit

1. What is data augmentation? Why is it needed?
Transform the data in different ways to help the model recognize different variations of the data.

1. What is the difference between `item_tfms` and `batch_tfms`?
`item_tfms` needs to do the transformation on each image individually
once all images are of same size transformations can be done using gpu using `batch_tfms`

1. What is a confusion matrix?
way to visualize output of the train/validation process

1. What does `export` save?
pickled moidel

1. What is it called when we use a model for getting predictions, instead of training?
inference

1. What are IPython widgets?
IPython widgets are used to create UI widgets in jupyter notebooks by combining javascript and python

1. When might you want to use CPU for deployment? When might GPU be better?
once model is trained, it's just a function that's being called and therefore if high parallelization isn't needed, cpu is better because it's cheaper and more widely available. gpu might be needed when high parallelization is needed.

1. What are the downsides of deploying your app to a server, instead of to a client (or edge) device such as a phone or PC?
- has to always be up and running instead of just when the client needs it
- managing machines
- security considerations transferring data back and forth


1. What are three examples of problems that could occur when rolling out a bear warning system in practice?
- not enough diversity images

1. What is "out-of-domain data"?
- data not present in the training set

1. What is "domain shift"?
- over time the domain may change

1. What are the three steps in the deployment process?


### Further Research

1. Consider how the Drivetrain Approach maps to a project or problem you're interested in.
1. When might it be best to avoid certain types of data augmentation?
1. For a project you're interested in applying deep learning to, consider the thought experiment "What would happen if it went really, really well?"
1. Start a blog, and write your first blog post. For instance, write about what you think deep learning might be useful for in a domain you're interested in.